In [ ]:
#Downloading libraries
!pip install imutils
!pip install opencv-python

In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

The Fashion MNIST dataset is identical to the MNIST dataset in terms of training set size, testing set size, 
number of class labels, and image dimensions:
60,000 training examples
10,000 testing examples
10 classes
28×28 grayscale images

In [2]:
#Ref https://www.pyimagesearch.com/2019/02/11/fashion-mnist-with-keras-and-deep-learning/
#Building the class
class Structure_CNN:
	@staticmethod
	def build(width, height, depth, classes): #Image parameters
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
        #Setting default value of Channel
		chanDim = -1
		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
        # first CONV => RELU => CONV => RELU => POOL layer set
        #Filter = 32
        #Kernel = 3,3
        # padding in order to make all sequences in a batch fit a given standard length, it is necessary to pad or truncate some sequences.
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
        #Activatation function
		model.add(Activation("relu"))
        #Batch normalization applies a transformation that maintains the mean output close to 0 
        #and the output standard deviation close to 1.
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
        #Dense implements the operation: output = activation(dot(input, kernel) + bias) 
        #where activation is the element-wise activation function passed as the activation argument, 
        #kernel is a weights matrix created by the layer, and 
        #bias is a bias vector created by the layer (only applicable if use_bias is True).
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5)) #The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time
		# softmax classifier
		model.add(Dense(classes))
        #For multi class output we are using softmax
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

In [3]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from sklearn.metrics import classification_report #Evaluation Matrix report
from tensorflow.keras.optimizers import SGD #Stocastic Gradient Boosting 
from tensorflow.keras.datasets import fashion_mnist #Dataset
from tensorflow.keras.utils import to_categorical #Transformation
from tensorflow.keras import backend as K #Interact with tensorflow in backend
from imutils import build_montages  #to show multiple images in opencv
import matplotlib.pyplot as plt
import numpy as np
import cv2 #For opencv
# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 2 #Number of epoch
INIT_LR = 1e-2 #Learning Rate
BS = 32 #Batch Size

In [4]:
# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
# if we are using "channels first" ordering, then reshape the design
# matrix such that the matrix is:
# 	num_samples x depth x rows x columns
#Channel_first means - Channel information is going in beginning after batch size
if K.image_data_format() == "channels_first":
	trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
	testX = testX.reshape((testX.shape[0], 1, 28, 28))
 
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the training and testing labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)
# initialize the names of the class
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

[INFO] loading Fashion MNIST...


In [5]:
# initialize the optimizer and model
print("[INFO] compiling model...")
#Applying Stocastic Gradient Descent
#momentum = method which helps accelerate gradients vectors in the right directions, thus leading to faster converging
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS) #Applies exponential decay to the learning rate. 
#depth 1 means greyscale
model = Structure_CNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training model...")
H = model.fit(x=trainX, y=trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...
[INFO] training model...
Epoch 1/2
1875/1875 [==============================] - 278s 148ms/step - loss: 0.5391 - accuracy: 0.8125 - val_loss: 0.3540 - val_accuracy: 0.8739
Epoch 2/2
1875/1875 [==============================] - 278s 148ms/step - loss: 0.3848 - accuracy: 0.8597 - val_loss: 0.3189 - val_accuracy: 0.8871


In [6]:
# make predictions on the test set
preds = model.predict(testX)
# show a nicely formatted classification report
print("[INFO] evaluating network...")
#argmax give the position where we get maximum value
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))


[INFO] evaluating network...
              precision    recall  f1-score   support

         top       0.84      0.85      0.84      1000
     trouser       0.99      0.97      0.98      1000
    pullover       0.81      0.84      0.83      1000
       dress       0.88      0.90      0.89      1000
        coat       0.79      0.82      0.80      1000
      sandal       0.99      0.95      0.97      1000
       shirt       0.71      0.65      0.68      1000
     sneaker       0.92      0.97      0.95      1000
         bag       0.97      0.97      0.97      1000
  ankle boot       0.96      0.95      0.96      1000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [7]:
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("D:\\RecogX_Backup\\EL\\DeepLearning\\plot.png")
plt.show()

<ipython-input-7-ca1993d21610>:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [8]:
# initialize our list of output images
images = []
# randomly select a few testing fashion items
for i in np.random.choice(np.arange(0, len(testY)), size=(16,)):
	# classify the clothing
	probs = model.predict(testX[np.newaxis, i])
	prediction = probs.argmax(axis=1)
	label = labelNames[prediction[0]]
 
	# extract the image from the testData if using "channels_first"
	# ordering
	if K.image_data_format() == "channels_first":
		image = (testX[i][0] * 255).astype("uint8")
 
	# otherwise we are using "channels_last" ordering
	else:
		image = (testX[i] * 255).astype("uint8")
        # initialize the text label color as green (correct)
	color = (0, 255, 0)
	# otherwise, the class label prediction is incorrect
	if prediction[0] != np.argmax(testY[i]):
		color = (0,0,255)

 
	# merge the channels into one image and resize the image from
	# 28x28 to 96x96 so we can better see it and then draw the
	# predicted label on the image
	image = cv2.merge([image] * 3)
	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
	cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
		color, 2)
	# add the image to our list of output images
	images.append(image)
# construct the montage for the images
montage = build_montages(images, (96, 96), (4, 4))[0]
# show the output montage
cv2.imshow("Fashion MNIST", montage)
cv2.waitKey(0)

-1